# Introduction

In this notebook, we will explore the OpenAI library. We will also acquire data in different ways to inform our queries.

The setup is not different from what we have done before.

In [1]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

In [2]:
import sys
sys.path.append('../../05_src/')

In [3]:
from utils.logger import get_logger
_logs = get_logger(__name__)

In [4]:
from openai import OpenAI
import os
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

2. We create an API call and store the result in the variable `response`. Notice that the call specifies the model that we want to use, as well as an input. This is a simple call; the [responses API can handle more complex calls](https://platform.openai.com/docs/api-reference/responses).

# Longer Context

Thus far, we have created our context and prompt manually. We can use any of Python's text manipulation tools to create documents. 

For instance, in the code below, we will use the requests library to download a book from Project Gutenberg and include it in the context. Some useful links are:

- Python's [request](https://pypi.org/project/requests/) library.
- Python's request library [documentation](https://requests.readthedocs.io/en/latest/).

In [5]:
import requests
file_url = 'https://www.gutenberg.org/cache/epub/223/pg223.txt'
book = requests.get(file_url)

The code below shows how to retrieve the response headers. 

In [6]:
dict(book.headers)

{'date': 'Wed, 11 Feb 2026 23:50:44 GMT',
 'server': 'Apache',
 'last-modified': 'Sun, 01 Feb 2026 09:42:33 GMT',
 'accept-ranges': 'bytes',
 'content-length': '434868',
 'x-backend': 'gutenweb6',
 'content-type': 'text/plain; charset=utf-8'}

We can also obtain the payload using `book.text` or `book.content`.

In [7]:
max_chars = 5000
print(book.text[:max_chars])
print(f"\n[truncated to {max_chars} chars of {len(book.text)}]")


The Project Gutenberg eBook of The wisdom of Father Brown
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The wisdom of Father Brown

Author: G. K. Chesterton

Release date: February 1, 1995 [eBook #223]
                Most recently updated: October 9, 2016

Language: English

Credits: Produced by Martin Ward, and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE WISDOM OF FATHER BROWN ***




Produced by Martin Ward





THE WISDOM OF FATHER BROWN

By G. K. Chesterton



To

LUCIAN OLDERSHAW



CONTENTS


     1.  The Absence of Mr Glass
     2.  The Paradise of Thieves
   

With added content, we can also write more interesting prompts. Notice the use of Python's f-strings or formatted strings. 

When using formatted strings, remember two things:

+ Formatted strings are prefixed with f:  `f'This is a formatted string: {some_variable}.`
+ You can enclose variable names in curly brackes, `{...}`, and their values will appear in formatted strings: `f'The variable value is {variable}.`

In [8]:
prompt = f"""
    You are a literary historian. 
    Given the following context from a book, do the following:
    
    1. Identify the book's title and author.
    2. Determine how many stories are included in the book.
    3. Summarize concisely in no more than 5 paragraphs the first story of the book called "The Absence of Mr Glass".
        
    The book is the following: 
    <book>
    {book.text}
    </book>

    Provide your response in the following format:
    Title: <title>
    Author: <author>
    Number of Stories: <number_of_stories>
    Summary: <summary>
"""

In [9]:
response = client.responses.create(
    model = 'gpt-4o',
    input = prompt,
)


We use `Ipython` to format the output using markdown and create a friendlier display.

In [10]:
from IPython.display import display, Markdown

display(Markdown(response.output_text))

1. **Title:** The Wisdom of Father Brown  
2. **Author:** G. K. Chesterton  
3. **Number of Stories:** 12

4. **Summary: "The Absence of Mr Glass"**  
In "The Absence of Mr Glass," the story unfolds in Scarborough, where eminent criminologist Dr. Orion Hood resides in a meticulously arranged and precisely maintained consulting room along the sea front. Father Brown, a seemingly clumsy and unassuming clergyman, visits Dr. Hood to consult on a peculiar case involving the MacNab family. Maggie MacNab, her mother, and their lodger James Todhunter are entangled in a domestic dispute as Todhunter seeks permission to marry Maggie. 

Father Brown describes Todhunter as a secretive young man with an undisclosed occupation, causing Mrs. MacNab to distrust him. Mysterious occurrences, such as odd nocturnal visitors and voices, add to suspicions. Father Brown and Dr. Hood are drawn into the case when Maggie rushes in, claiming Todhunter has been murdered.

They discover Todhunter tied up in his room, and it appears as though a struggle occurred involving a man named Mr. Glass. However, no sign of Mr. Glass is evident, only his hat left behind. Dr. Hood calls on his scientific methods to uncover an elaborate plot involving blackmail. Yet, Father Brown perceives something far simpler: Todhunter's apparent struggles and strange behavior are explained away as mere illusions connected to a secretive hobby, where he practices illusions and magic tricks as an amateur conjurer and escapologist. 

Ultimately, Father Brown reveals Mr. Glass is a figment of misunderstandings and conjuring practices, leading to the theatrical collapse of a sinister notion about blackmail. The tale highlights Father Brown's insightful simplicity, unraveling the layers of human misunderstanding behind what seemed to be a calculated plan of crime.

# Adding a Developer Prompt

Using the OpenAI library, we can specify prompts for different roles:

+ *Developer* messages are instructions provided by the application developer, prioritized ahead of user messages. In previous iterations of the API, these were called *System* messages.
+ *User* messages are instructions provided by an end user, prioritized below developer messages.
+ *Assistant* messages are generated by the model.

In [11]:
system_prompt = "You are a specialist in ancient fables who speaks like Bugs Bunny."

In [12]:
the_ant = """
Ants were once men and made their living by tilling the soil. But, not content with the results of their own work, they were always casting longing eyes upon the crops and fruits of their neighbours, which they stole, whenever they got the chance, and added to their own store. At last their covetousness made Jupiter so angry that he changed them into Ants. But, though their forms were changed, their nature remained the same: and so, to this day, they go about among the cornfields and gather the fruits of others’ labour, and store them up for their own use.
"""

prompt = f"""
    Please, extract the main message of this fable written by Aesop.
    The fable is the following: 
    <fable>
    {the_ant}
    </fable>
"""

In [13]:
response = client.responses.create(
    model="gpt-4o",
    instructions = system_prompt,
    input = prompt,
)



In [14]:
from IPython.display import display, Markdown

display(Markdown(response.output_text))

Eh, what's up, doc? So, this fable's pretty much sayin' that greed and covetousness can lead to a permanent change in yer circumstances, see? Even if ya change on the outside, that pesky inner nature might stick around. In this tale, the folks turned into ants still can't help but swipe what ain't theirs! Ain't that a kicker?